# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 13 2023 8:30AM,257186,10,9142099,"Digital Brands, LLC",Released
1,Feb 13 2023 8:30AM,257186,10,9142101,"Digital Brands, LLC",Released
2,Feb 13 2023 8:30AM,257186,10,9142101,"Digital Brands, LLC",Released
3,Feb 13 2023 8:30AM,257179,10,PRF-52564,Bio-PRF,Released
4,Feb 11 2023 9:47AM,257190,22,641198,"NBTY Global, Inc.",Released
5,Feb 11 2023 9:47AM,257189,22,641165,"NBTY Global, Inc.",Released
6,Feb 11 2023 9:46AM,257188,22,641055,"NBTY Global, Inc.",Released
7,Feb 10 2023 3:13PM,257181,16,9141507,Sartorius Bioprocess Solutions,Executing
8,Feb 10 2023 3:13PM,257181,16,9141509,Sartorius Bioprocess Solutions,Executing
9,Feb 10 2023 3:08PM,257180,19,ELI02142023,Eli Lilly and Company,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
24,257181,Executing,2
25,257186,Released,3
26,257188,Released,1
27,257189,Released,1
28,257190,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
257181,NaN,2.0,NaN
257186,NaN,NaN,3.0
257188,NaN,NaN,1.0
257189,NaN,NaN,1.0
257190,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257130,0.0,0.0,15.0
257134,0.0,0.0,1.0
257137,0.0,0.0,1.0
257138,0.0,0.0,1.0
257139,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
257130,0,0,15
257134,0,0,1
257137,0,0,1
257138,0,0,1
257139,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257130,0,0,15
1,257134,0,0,1
2,257137,0,0,1
3,257138,0,0,1
4,257139,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,257130,,,15
1,257134,,,1
2,257137,,,1
3,257138,,,1
4,257139,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 13 2023 8:30AM,257186,10,"Digital Brands, LLC"
3,Feb 13 2023 8:30AM,257179,10,Bio-PRF
4,Feb 11 2023 9:47AM,257190,22,"NBTY Global, Inc."
5,Feb 11 2023 9:47AM,257189,22,"NBTY Global, Inc."
6,Feb 11 2023 9:46AM,257188,22,"NBTY Global, Inc."
7,Feb 10 2023 3:13PM,257181,16,Sartorius Bioprocess Solutions
9,Feb 10 2023 3:08PM,257180,19,Eli Lilly and Company
10,Feb 10 2023 2:49PM,257178,19,"Emersa Waterbox, LLC"
12,Feb 10 2023 2:31PM,257177,19,"GUSA Granules USA, Inc."
13,Feb 10 2023 2:11PM,257175,10,ISDIN Corporation


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Feb 13 2023 8:30AM,257186,10,"Digital Brands, LLC",,,3
1,Feb 13 2023 8:30AM,257179,10,Bio-PRF,,,1
2,Feb 11 2023 9:47AM,257190,22,"NBTY Global, Inc.",,,1
3,Feb 11 2023 9:47AM,257189,22,"NBTY Global, Inc.",,,1
4,Feb 11 2023 9:46AM,257188,22,"NBTY Global, Inc.",,,1
5,Feb 10 2023 3:13PM,257181,16,Sartorius Bioprocess Solutions,,2,
6,Feb 10 2023 3:08PM,257180,19,Eli Lilly and Company,,,1
7,Feb 10 2023 2:49PM,257178,19,"Emersa Waterbox, LLC",,,2
8,Feb 10 2023 2:31PM,257177,19,"GUSA Granules USA, Inc.",,,1
9,Feb 10 2023 2:11PM,257175,10,ISDIN Corporation,,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 8:30AM,257186,10,"Digital Brands, LLC",3,,
1,Feb 13 2023 8:30AM,257179,10,Bio-PRF,1,,
2,Feb 11 2023 9:47AM,257190,22,"NBTY Global, Inc.",1,,
3,Feb 11 2023 9:47AM,257189,22,"NBTY Global, Inc.",1,,
4,Feb 11 2023 9:46AM,257188,22,"NBTY Global, Inc.",1,,
5,Feb 10 2023 3:13PM,257181,16,Sartorius Bioprocess Solutions,,2,
6,Feb 10 2023 3:08PM,257180,19,Eli Lilly and Company,1,,
7,Feb 10 2023 2:49PM,257178,19,"Emersa Waterbox, LLC",2,,
8,Feb 10 2023 2:31PM,257177,19,"GUSA Granules USA, Inc.",1,,
9,Feb 10 2023 2:11PM,257175,10,ISDIN Corporation,1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 8:30AM,257186,10,"Digital Brands, LLC",3,,
1,Feb 13 2023 8:30AM,257179,10,Bio-PRF,1,,
2,Feb 11 2023 9:47AM,257190,22,"NBTY Global, Inc.",1,,
3,Feb 11 2023 9:47AM,257189,22,"NBTY Global, Inc.",1,,
4,Feb 11 2023 9:46AM,257188,22,"NBTY Global, Inc.",1,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 8:30AM,257186,10,"Digital Brands, LLC",3.0,NaN,NaN
1,Feb 13 2023 8:30AM,257179,10,Bio-PRF,1.0,NaN,NaN
2,Feb 11 2023 9:47AM,257190,22,"NBTY Global, Inc.",1.0,NaN,NaN
3,Feb 11 2023 9:47AM,257189,22,"NBTY Global, Inc.",1.0,NaN,NaN
4,Feb 11 2023 9:46AM,257188,22,"NBTY Global, Inc.",1.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Feb 13 2023 8:30AM,257186,10,"Digital Brands, LLC",3.0,0.0,0.0
1,Feb 13 2023 8:30AM,257179,10,Bio-PRF,1.0,0.0,0.0
2,Feb 11 2023 9:47AM,257190,22,"NBTY Global, Inc.",1.0,0.0,0.0
3,Feb 11 2023 9:47AM,257189,22,"NBTY Global, Inc.",1.0,0.0,0.0
4,Feb 11 2023 9:46AM,257188,22,"NBTY Global, Inc.",1.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,1800159,33.0,0.0,0.0
15,771478,3.0,1.0,8.0
16,1542871,5.0,2.0,0.0
19,1285840,6.0,0.0,0.0
20,257171,1.0,0.0,0.0
21,514325,2.0,0.0,0.0
22,771567,3.0,0.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,1800159,33.0,0.0,0.0
1,15,771478,3.0,1.0,8.0
2,16,1542871,5.0,2.0,0.0
3,19,1285840,6.0,0.0,0.0
4,20,257171,1.0,0.0,0.0
5,21,514325,2.0,0.0,0.0
6,22,771567,3.0,0.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,33.0,0.0,0.0
1,15,3.0,1.0,8.0
2,16,5.0,2.0,0.0
3,19,6.0,0.0,0.0
4,20,1.0,0.0,0.0
5,21,2.0,0.0,0.0
6,22,3.0,0.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,33.0
1,15,Released,3.0
2,16,Released,5.0
3,19,Released,6.0
4,20,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21,22
Status,,,,,,,
Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0
Executing,0.0,1.0,2.0,0.0,0.0,0.0,0.0
Released,33.0,3.0,5.0,6.0,1.0,2.0,3.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21,22
0,Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,2.0,0.0,0.0,0.0,0.0
2,Released,33.0,3.0,5.0,6.0,1.0,2.0,3.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21,22
0,Completed,0.0,8.0,0.0,0.0,0.0,0.0,0.0
1,Executing,0.0,1.0,2.0,0.0,0.0,0.0,0.0
2,Released,33.0,3.0,5.0,6.0,1.0,2.0,3.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()